In [1]:
import os
import numpy as np, pandas as pd
import json

In [2]:
pd.options.display.max_rows = 100
# unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
# users = pd.read_csv('ml-latest/users.csv', sep=',',
#  header=None, names=unames, engine='python', encoding='latin-1')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::',
 header=None, names=rnames, engine='python', encoding='latin-1')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::',
 header=None, names=mnames, engine='python', encoding='latin-1')

In [10]:
movies[['movie_id', 'title']].to_json('movie_map.json', orient='records')

In [3]:
ratings = ratings[ratings["rating"] >= 5]  # Remove disliked movies
ratings = ratings.merge(movies, on="movie_id")  # Add in titles
merged_ratings = pd.merge(ratings[['user_id', 'movie_id', 'rating']], movies[['movie_id', 'title','genres']], on='movie_id')
merged_ratings = merged_ratings.drop_duplicates(subset=['user_id', 'title'])

In [4]:
count =0 
instruction = "Given a list of liked movies, recommend 4 more movies the user would like."
prompt_list = []
for _, user in merged_ratings.groupby("user_id"): 
   
        if user.shape[0] >= 10:
                count +=1
                
                # Get amount of 10 movie blocks in user.shape[0]
                num_blocks = int(user.shape[0]/10)
                # For each block, create a fine-tuning example
                for i in range(num_blocks):
                        # Get 10 movies
                        movies = user.iloc[i*10:(i+1)*10]
                        # fitst 6 movies are the input, last 4 are the output in string form
                        input_str = movies.iloc[:6].title.str.cat(sep=', ')
                        output_str = movies.iloc[6:].title.str.cat(sep=', ')
                        
                        prompt = {
                                "instruction": f"{instruction}",
                                "input": f"Movies user liked: {input_str}. Recommendation:",
                                "output": f"{output_str}"
                                }
                        prompt_list.append(prompt)


                        
        
data_len = len(prompt_list)           
                
print('Prepared fine-tuning and evaluation datasets')     
print(f'Tuning set is of size {data_len}, given even amount for train/eval, the respective sizes of sets are {int(data_len*0.85)} for training and {int(data_len*0.15)} for testing')

Prepared fine-tuning and evaluation datasets
Tuning set is of size 19906, given even amount for train/eval, the respective sizes of sets are 16920 for training and 2985 for testing


In [5]:
with open('llama_train_red_wh.json', 'w') as outfile:
    json.dump(prompt_list, outfile)

In [6]:
with open('llama_train_adj.json', 'w') as outfile:
    json.dump(prompt_list[:int(data_len*0.85)], outfile)

with open('llama_eval_adj.json', 'w') as outfile:
    json.dump(prompt_list[int(data_len*0.85):], outfile)

In [8]:
movie_names = merged_ratings['title'].unique()


In [9]:
counter = 0
movie_mapping_dict = {}
for movie in movie_names:
    
    movie_mapping_dict[movie] = counter 
    print(f"{movie}::ID::{counter}")
    counter+=1

One Flew Over the Cuckoo's Nest (1975)::ID::0
Bug's Life, A (1998)::ID::1
Ben-Hur (1959)::ID::2
Christmas Story, A (1983)::ID::3
Beauty and the Beast (1991)::ID::4
Sound of Music, The (1965)::ID::5
Awakenings (1990)::ID::6
Back to the Future (1985)::ID::7
Schindler's List (1993)::ID::8
Pocahontas (1995)::ID::9
Last Days of Disco, The (1998)::ID::10
Cinderella (1950)::ID::11
Apollo 13 (1995)::ID::12
Toy Story (1995)::ID::13
Rain Man (1988)::ID::14
Mary Poppins (1964)::ID::15
Dumbo (1941)::ID::16
Saving Private Ryan (1998)::ID::17
Shine (1996)::ID::18
Few Good Men, A (1992)::ID::19
Hustler, The (1961)::ID::20
Gladiator (2000)::ID::21
Hunt for Red October, The (1990)::ID::22
Simon Birch (1998)::ID::23
Stand by Me (1986)::ID::24
Green Mile, The (1999)::ID::25
Gandhi (1982)::ID::26
Braveheart (1995)::ID::27
Close Encounters of the Third Kind (1977)::ID::28
On the Waterfront (1954)::ID::29
Amadeus (1984)::ID::30
Remains of the Day, The (1993)::ID::31
Jurassic Park (1993)::ID::32
Doctor Zhiva

In [10]:
with open('movie_map.json', 'w') as outfile:
    json.dump(movie_mapping_dict, outfile)